In [1]:
import warnings
import json
from cProfile import label
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
# from sklearnex import patch_sklearn  ## 加速svm
# patch_sklearn("SVC")
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_curve,roc_auc_score,recall_score,precision_recall_curve,auc,precision_score,f1_score,class_likelihood_ratios,accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.multiclass import OneVsRestClassifier
from sklearn.utils import resample,shuffle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
from learn_curving_fold import plot_learning_curves
from imblearn.over_sampling import RandomOverSampler
import shap
import time
import copy
#from svm-gpu import SVM
from IPython.display import display
import os
import joblib
from tqdm import trange
from keras.layers import Dense, Conv2D, AveragePooling2D, MaxPooling2D, Flatten, Conv1D, AveragePooling1D, BatchNormalization
from keras.layers.regularization.dropout import Dropout
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers

d:\CODE\KD\kdenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [58]:
def kfold_self_score_up(
    cv = 5,
    model_path =None,
    path = 'k_fold_train_validation_42',
    random_state = 30,
    ):
    '''
    cv : inter, default:5 -- input how many fold we want to do
    model_path:default: None -- input the path that is been trained model
    path: the folder for the datas
    '''
    specificity_list = []
    f1_list = []
    recall_list = []
    precision_list = []
    positive_list = []
    negative_list = []
    score = pd.DataFrame(columns=[])

    for i in trange(1,cv+1):
        score[f'{i}'] = []
        fold_train = pd.read_csv(path+'/{}-fold_train_z.csv'.format(i))
        fold_test = pd.read_csv(path+'/{}-fold_test_z.csv'.format(i))
        #rename(消去columns項內非英文以及數字的解)
        try:
            fold_train = fold_train.drop(columns=['Unnamed: 0'])
            fold_test = fold_test.drop(columns=['Unnamed: 0'])
        except:
            pass
        fold_train = fold_train.rename(columns=lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
        fold_test = fold_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
        ## upsampling train set 
        one_message = fold_train[fold_train["Class"] == 1 ]
        zero_message =fold_train[fold_train["Class"] == 0]
        one_message_up = resample(one_message,
                    replace=True,
                    n_samples=len(zero_message),
                    random_state=42)
        fold_train = pd.concat([one_message_up,zero_message])
        fold_train = shuffle(fold_train,random_state = random_state)
        fold_train= fold_train.reset_index(drop = True)
        # set the train set for single fold
        x_train = fold_train.drop(columns=['Class'])
        y_train = fold_train['Class']       
        ## set the non upsampling validation for the single fold
        one_message_test = fold_test[fold_test["Class"] == 1 ]
        zero_message_test =fold_test[fold_test["Class"] == 0]
        one_message_up_test = resample(one_message_test,
                    replace=True,
                    n_samples=len(zero_message_test),
                    random_state=42)
        fold_test = pd.concat([one_message_up_test,zero_message_test])
        fold_test = shuffle(fold_test,random_state = random_state)
        fold_test= fold_test.reset_index(drop = True)         
        x_test = fold_test.drop(columns=["Class"])
        y_test = fold_test['Class']
        '============================================================='
        model = joblib.load(model_path+f'/{i}-model')
        y_pred = model.predict(x_test)
        f1_list.append(f1_score(y_test, y_pred))
        specificity_list.append(recall_score(y_test, y_pred, pos_label=0))
        recall_list.append(recall_score(y_test, y_pred, pos_label=1))
        precision_list.append(precision_score(y_test, y_pred))
        likelihood = class_likelihood_ratios(y_test,y_pred)
        positive_list.append(likelihood[0])
        negative_list.append(likelihood[1])
    score.loc['f1'] = f1_list
    score.loc['precision'] = precision_list
    score.loc['recall'] = recall_list
    score.loc['specificity'] = specificity_list        
    score.loc['positive likelihood']  = positive_list
    score.loc['negative likelihood'] = negative_list
    score['average'] = score.sum(axis = 1)/cv
    return(score)

In [59]:
model = 'lgbm'
path = "D:/CODE/KD/code_after_web/data/Top5_under7(ByKeptGPT,with monocyte count)/k-fold/30" 
model_path = f'D:/CODE/KD/code_after_web/output/Under 7/Top5_under7(ByKeptGPT,with monocyte count)/{model}'
model_save_path = f'D:/CODE/KD/code_after_web/output/Under 7/Top5_under7(ByKeptGPT,with monocyte count)/{model}'
score_up = kfold_self_score_up(path=path,model_path=model_path,random_state=30)
score_up.to_csv(f'{model_save_path}/{model}_score_up_spe_validation_likelihood.csv')
score_up

  0%|          | 0/5 [00:00<?, ?it/s]Trying to unpickle estimator LabelEncoder from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
 20%|██        | 1/5 [00:00<00:00,  6.08it/s]Trying to unpickle estimator LabelEncoder from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
 40%|████      | 2/5 [00:00<00:00,  6.51it/s]Trying to unpickle estimator LabelEncoder from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
 60%|██████    | 3

,1,2,3,4,5,average
f1,0.884816,0.874482,0.882769,0.877766,0.870515,0.878069
precision,0.906379,0.900272,0.899375,0.904743,0.897003,0.901554
recall,0.864255,0.850128,0.866765,0.852351,0.845547,0.855809
specificity,0.910730,0.905826,0.903024,0.910260,0.902911,0.906550
positive likelihood,9.681319,9.027237,8.937896,9.497954,8.709022,9.170685
negative likelihood,0.149051,0.165453,0.147543,0.162206,0.171062,0.159063


In [60]:
def kfold_self_score(
    cv = 5,
    model_path =None,
    path = 'k_fold_train_validation_42',
    random_state = 30,
    ):
    '''
    cv : inter, default:5 -- input how many fold we want to do
    model_path:default: None -- input the path that is been trained model
    path: the folder for the datas
    '''
    specificity_list = []
    f1_list = []
    recall_list = []
    precision_list = []
    positive_list = []
    negative_list = []
    accuracy = []
    score = pd.DataFrame(columns=[])

    for i in trange(1,cv+1):
        score[f'{i}'] = []
        fold_train = pd.read_csv(path+'/{}-fold_train_z.csv'.format(i))
        fold_test = pd.read_csv(path+'/{}-fold_test_z.csv'.format(i))
        #rename(消去columns項內非英文以及數字的解)
        try:
            fold_train = fold_train.drop(columns=['Unnamed: 0'])
            fold_test = fold_test.drop(columns=['Unnamed: 0'])
        except:
            pass
        fold_train = fold_train.rename(columns=lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
        fold_test = fold_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
         ## upsampling train set 
        one_message = fold_train[fold_train["Class"] == 1 ]
        zero_message =fold_train[fold_train["Class"] == 0]
        one_message_up = resample(one_message,
                    replace=True,
                    n_samples=len(zero_message),
                    random_state=42)
        fold_train = pd.concat([one_message_up,zero_message])
        fold_train = shuffle(fold_train,random_state = random_state)
        fold_train= fold_train.reset_index(drop = True)
        ## set the train set for single fold
        x_train = fold_train.drop(columns=['Class'])
        y_train = fold_train['Class']
        '=================================================================================='
        ## set the non upsampling validation for the single fold
        fold_test = shuffle(fold_test,random_state = random_state)
        fold_test= fold_test.reset_index(drop = True)
        x_test = fold_test.drop(columns=["Class"])
        y_test = fold_test['Class']
        '============================================================='
        model = joblib.load(model_path+f'/{i}-model')
        y_pred = model.predict(x_test)
        f1_list.append(f1_score(y_test, y_pred))
        specificity_list.append(recall_score(y_test, y_pred, pos_label=0))
        recall_list.append(recall_score(y_test, y_pred, pos_label=1))
        precision_list.append(precision_score(y_test, y_pred))
        likelihood = class_likelihood_ratios(y_test,y_pred)
        accuracy.append(accuracy_score(y_test,y_pred))
        positive_list.append(likelihood[0])
        negative_list.append(likelihood[1])
    score.loc['f1'] = f1_list
    score.loc['precision'] = precision_list
    score.loc['recall'] = recall_list
    score.loc['specificity'] = specificity_list      
    score.loc['positive likelihood']  = positive_list
    score.loc['negative likelihood'] = negative_list
    score.loc['accuracy'] = accuracy
    score['average'] = score.sum(axis = 1)/cv
    return(score)

In [61]:
model = 'lgbm'
path = "D:/CODE/KD/code_after_web/data/Top5_under7(ByKeptGPT,with monocyte count)/k-fold/30" 
model_path = f'D:/CODE/KD/code_after_web/output/Under 7/Top5_under7(ByKeptGPT,with monocyte count)/{model}'
model_save_path = f'D:/CODE/KD/code_after_web/output/Under 7/Top5_under7(ByKeptGPT,with monocyte count)/{model}'
score_up = kfold_self_score(path=path,model_path=model_path,random_state=30)
score_up.to_csv(f'{model_save_path}/{model}_score_spe_validation_likelihood.csv')
score_up

  0%|          | 0/5 [00:00<?, ?it/s]Trying to unpickle estimator LabelEncoder from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
Trying to unpickle estimator LabelEncoder from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
 40%|████      | 2/5 [00:00<00:00, 15.45it/s]Trying to unpickle estimator LabelEncoder from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
Trying to unpickle estimator LabelEncoder from version 1.0.2 wh

,1,2,3,4,5,average
f1,0.419865,0.391660,0.396802,0.415292,0.407067,0.406137
precision,0.277061,0.254352,0.257062,0.274257,0.267907,0.266128
recall,0.866460,0.851133,0.869427,0.854938,0.847059,0.857803
specificity,0.910730,0.905826,0.903024,0.910260,0.902911,0.906550
positive likelihood,9.706014,9.037903,8.965344,9.526788,8.724598,9.192129
negative likelihood,0.146630,0.164344,0.144596,0.159363,0.169387,0.156864


In [4]:
def kfold_test_score(
    best_model,
    model_path =None,
    file_test = 'k_fold_train_validation_42',
    random_state = 30,
    ):
    '''
    cv : inter, default:5 -- input how many fold we want to do
    model_path:default: None -- input the path that is been trained model
    path: the folder for the datas
    '''
    specificity_list = []
    f1_list = []
    recall_list = []
    precision_list = []
    positive_list = []
    negative_list = []
    accuracy = []

    score = pd.DataFrame(columns=[])
    score['test'] = []
    fold_test = pd.read_csv(file_test)
    #rename(消去columns項內非英文以及數字的解)
    try:
        fold_test = fold_test.drop(columns=['Unnamed: 0'])
    except:
        pass
    '=================================================================================='
    ## set the non upsampling validation for the single fold
    fold_test = shuffle(fold_test,random_state = random_state)
    fold_test= fold_test.reset_index(drop = True)
    x_test = fold_test.drop(columns=["Class"])
    y_test = fold_test['Class']
    '============================================================='
    model = joblib.load(model_path+f'/{best_model}-model')
    y_pred = model.predict(x_test)
    f1_list.append(f1_score(y_test, y_pred))
    specificity_list.append(recall_score(y_test, y_pred, pos_label=0))
    recall_list.append(recall_score(y_test, y_pred, pos_label=1))
    precision_list.append(precision_score(y_test, y_pred))
    likelihood = class_likelihood_ratios(y_test,y_pred)
    positive_list.append(likelihood[0])
    negative_list.append(likelihood[1])
    accuracy.append(accuracy_score(y_test,y_pred))

    score.loc['f1'] = f1_list
    score.loc['precision'] = precision_list
    score.loc['recall'] = recall_list
    score.loc['specificity'] = specificity_list      
    score.loc['positive likelihood']  = positive_list
    score.loc['negative likelihood'] = negative_list
    score.loc['accuracy'] = accuracy

    return(score)

In [5]:
model = 'lgbm'
best_model_num = 3
model_path = f'D:/CODE/KD/code_after_web/output/Under 7/KeptGPT_under7(ByKeptGPT,with monocyte count)/{model}'
file_test = f'D:/CODE/KD/code_after_web/data/KeptGPT_under7(ByKeptGPT)/k-fold_monocyte_count/30/total-test_z-fold{best_model_num}.csv' 
model_save_path = f'D:/CODE/KD/code_after_web/output/Under 7/Top5_under7(ByKeptGPT,with monocyte count)/{model}'
score_test = kfold_test_score(model_path = model_path,file_test=file_test,random_state = 30,best_model = best_model_num)
##score_test.to_csv(f'{model_save_path}/score_test_likelihood.csv')
score_test

Trying to unpickle estimator LabelEncoder from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations


,test
f1,0.451520
precision,0.302913
recall,0.886364
specificity,0.920610
positive likelihood,11.164725
negative likelihood,0.123436
accuracy,0.919327


In [56]:
recall = 0.897727272727272
specificity = 0.91640866873065
print("Positive LR",recall/(1-specificity))
print("Negative LR",(1-recall)/specificity)

Positive LR 10.739478114478082
Negative LR 0.11160165847665925


In [7]:
# precision = 0.138009049773755
# recall = 0.693181818181818
# specificity = 0.831490490933215
# TP = 1*recall
# FN = 1-TP
# FP = (TP-TP*precision)/precision
# TN = (FP*specificity)/(1-specificity)

TP=139
FN=27
FP=6
TN=142

PPV = (TP)/(TP+FP)
NPV = (TN)/(FN+TN) 

FPR = (FP)/(TN+FP)
TPR = (TP)/(TP+FN)
FNR = (FN)/(TP+FN)
TNR = (TN)/(TN+FP)
accuracy = (TN+TP)/(TN+TP+FN+FP)

print('PPV:',PPV,'NPV:',NPV)
print('P_LR:',TPR/FPR,'N_LR:',FNR/TNR)
print('recall:',TPR,'specificity:',TNR,'precision:',PPV)
print('accuracy',accuracy)

PPV: 0.9939393939393939 NPV: 0.1590909090909091
P_LR: 6.527363184079602 N_LR: 0.21037668798862827
recall: 0.8159203980099502 specificity: 0.875 precision: 0.9939393939393939
accuracy 0.8181818181818182
